In [1]:
import pandas as pd
import random
import re
import codecs

In [64]:
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [5]:
from pyltp import Segmentor
from pyltp import Postagger
from pyltp import SementicRoleLabeller
from pyltp import NamedEntityRecognizer
from pyltp import Parser

In [6]:
with codecs.open('data/orig.org_order.txt',encoding='utf-8') as f:
    orig=[]
    for line in f:
        orig.append(line.strip())
with codecs.open('data/manual.del.star.6584.txt',encoding='utf-8') as f:
    manual_del=[]
    for line in f:
        manual_del.append(line.strip())
with codecs.open('data/answers.6584.txt',encoding='utf-8') as f:
    answer_tp=[]
    for line in f:
        answer_tp.append(line.strip())

In [7]:
print(orig.__len__(),manual_del.__len__(),answer_tp.__len__())

6584 6584 6584


In [8]:
sent_data=[]
for i in range(len(orig)):
    row=[orig[i],manual_del[i],answer_tp[i]]
    sent_data.append(row)
print(sent_data.__len__())
sent_data_pd=pd.DataFrame(sent_data,columns=['orig','manual_del','ans_tp'],index=None)

6584


In [9]:
sent_data_pd[24:34]

,orig,manual_del,ans_tp
24,不仅外观上略显累赘,外观 累赘\t*****\t不仅\t*****\t上略显\t*****,不仅<Target>外观</Target>上略显<P>累赘</P>
25,所以在其它相机上常见的紫边现像已经很少了,紫边现像 很少了\t*****\t所以在其它相机上常见的\t*****\t已经\t*****,所以在其它相机上常见的<Target>紫边</Target>现像已经很<P>少</P>了
26,精良的做工,精 良 的 做 工 \t*****,<P>精良</P>的<Target>做工</Target>
27,而且配置了接近完美的手动功能人像,配置了接近完美的手动功能人像\t*****\t而且\t*****,而且配置了接近<P>完美</P>的<Target>手动功能</Target>人像
28,快门延时很长,快 门 延 时 很 长 \t*****,<Target>快门延时</Target>很<P>长</P>
29,使得拍照的抓拍能力不是很强,拍照的抓拍能力不是很强\t*****\t使得\t*****,使得<Target>拍照的抓拍能力</Target>不是很<P>强</P>
30,非常清晰的取景器图像,非 常 清 晰 的 取 景 器 图 像 \t*****,非常<P>清晰</P>的<Target>取景器图像</Target>
31,白平衡控制方面也不出众,白平衡控制 也不出众\t*****\t方面\t*****,<Target>白平衡</Target>控制方面也不<P>出众</P>
32,同时小巧的高速快门,小巧的高速快门\t*****\t同时\t*****,同时小巧的<P>高速</P><Target>快门</Target>
33,机顶闪光灯位置相当合理,机 顶 闪 光 灯 位 置 相 当 合 理 \t*****,机顶<Target>闪光灯</Target>位置相当<P>合理</P>


In [12]:
segmentor = Segmentor()  # 初始化实例
segmentor.load('../ltp_data/cws.model')  # 加载模型
postagger = Postagger() # 初始化实例
postagger.load('../ltp_data/pos.model')  # 加载模型
parser = Parser() # 初始化实例
parser.load('../ltp_data/parser.model')  # 加载模型

# print ("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
# parser.release()  # 释放模型
def segment_pos(sentence):
    words = segmentor.segment(sentence)  # 分词
    words_list = list(words)
    postags = postagger.postag(words)  # 词性标注
    word_pos=[]
    arcs = parser.parse(words, postags)  # 句法分析
    for it in zip(words_list,postags,arcs):
        word_pos.append((it[0],it[1],it[2].head,it[2].relation))
#     print ("\t".join("%d:%s" % (arc.head, arc.relation) for arc in arcs))
    
    # postagger.release()  # 释放模型
    # segmentor.release()  # 释放模型
    return word_pos

In [13]:
segment_pos('所以在其它相机上常见的紫边现像已经很少了')

[('所以', 'c', 12, 'ADV'),
 ('在', 'p', 6, 'ADV'),
 ('其它', 'r', 4, 'ATT'),
 ('相机', 'n', 5, 'ATT'),
 ('上', 'nd', 2, 'POB'),
 ('常见', 'a', 8, 'ATT'),
 ('的', 'u', 6, 'RAD'),
 ('紫边', 'n', 9, 'ATT'),
 ('现像', 'n', 12, 'SBV'),
 ('已经', 'd', 12, 'ADV'),
 ('很', 'd', 12, 'ADV'),
 ('少', 'a', 0, 'HED'),
 ('了', 'u', 12, 'RAD')]

In [14]:
orig_seg=sent_data_pd['orig'].apply(lambda x: segment_pos(x))

In [15]:
sent_data_pd['orig_seg']=orig_seg

In [16]:
def get_labels(row):
    items=row['manual_del'].replace('\t','').split('*****')[:-1]
    orig_seg=row['orig_seg']
    new_orig_seg=[]
#     word=[it[0] for it in orig_seg]
    if len(items)>1:
        will_del='\t'.join(items[1:])
        for word in orig_seg:
            if word[0] in will_del:
                new_orig_seg.append((word[0],word[1],word[2],word[3],'Y'))
            else:
                new_orig_seg.append((word[0],word[1],word[2],word[3],'N'))
    else:
        for word in orig_seg:
                new_orig_seg.append((word[0],word[1],word[2],word[3],'N'))
#     print(items,new_orig_seg)
    return new_orig_seg

In [17]:
labels=[]
for i in range(sent_data_pd.shape[0]):
    labels.append(get_labels(sent_data_pd.loc[i]))
sent_data_pd['labels']=labels

In [18]:
with codecs.open('resource/perception.txt',encoding='utf-8') as f:
    percept_words=[line.strip() for line in f]
percept_words=set(percept_words)
def perception(word):
    if word in percept_words:
        return True
    else:
        return False

In [19]:
with codecs.open('resource/polarity_words_1.txt',encoding='utf-8') as f:
    polar_words1=[line.strip() for line in f]
with codecs.open('resource/polarity_words_2.txt',encoding='utf-8') as f:
    polar_words2=[line.strip() for line in f]
polar_words1.extend(polar_words2)
polar_word=set(polar_words1)
def polarword(word):
    if word in polar_word:
        return True
    else:
        return False

In [20]:
import json
with open('resource/word2cluster.json') as f:
    word2cluster=json.load(f)

In [21]:
def word_cluster(word):
    try:
        cluster=word2cluster[word]
    except KeyError:
        cluster='0000'
    return cluster

In [23]:
with codecs.open('resource/baike-50.vec.txt',encoding='utf-8') as f:
    word2emb={}
    for line in f:
        items=line.strip().split()
        emb=[float(it) for it in items[1:]]
        try:
            word2emb[items[0]]=emb
        except:
            pass       

In [24]:
def add_embed(word):
    embed={}
    try:
        for ind,it in enumerate(word2emb[word]):
            embed[str(ind)]=it
    except:
        pass
    return embed

In [25]:
def word2feature(sent,i):
    sent_len=len(sent)
    word=sent[i][0]
    postag=sent[i][1]
    features={
        'word':word,
        'pref:word':word[0],
        'suff:word':word[-1],
        'postag': postag,
        'perception':perception(word),
        'polar_word':polarword(word),
        'cluster':word_cluster(word),

        # negative effect    
#         'word_head':sent[i][2],
#         'word_relation':sent[i][3],
    }
    if i>0 and i<sent_len-1:
        # about word
        word1=sent[i-1][0]
        word2=sent[i+1][0]
        features.update({
            '-1:word':word1,
            '+1:word':word2,
            '-1+word':word1+'|'+word,
            '+1+word':word+'|'+word2,
        })
        # about tag
        postag1=sent[i-1][1]
        postag2=sent[i+1][1]
        features.update({
            '-1:postag': postag1,
            '+1:postag': postag2,
            '-1+postag':postag1+'|'+postag,
            '+1+posttag':postag+'|'+postag2,
        })
    elif i==0 and len(sent)>1:
        # about word
        word2=sent[i+1][0]
        features.update({
            '+1:word':word2,
            '+1+word':word+'|'+word2,
        })
        # about tag
        postag2=sent[i+1][1]
        features.update({
            '+1:postag': postag2,
            '+1+posttag':postag+'|'+postag2,
        })
        features['EOS']=True
    elif len(sent)>1:
        # about word
        word1=sent[i-1][0]
        features.update({
            '-1:word':word1,
            '-1+word':word1+'|'+word,
        })
        # about tag
        postag1=sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1+postag':postag1+'|'+postag,
        })
        features['END']=True
    else:
        features['EOS']=True
        features['END']=True
    features.update(add_embed(word))
    return features
def sent2features(sent):
    return [word2feature(sent,i) for i in range(len(sent))]
def sent2labels(sent):
    return [it[4] for it in sent]
def sent2token(sent):
    return [it[0] for it in sent]

In [38]:
percent=0.9
data_num=sent_data_pd.shape[0]
train_num=int(data_num*percent)
random.shuffle(labels)
train_sents=labels[:train_num]
test_sents=labels[train_num:]
x_train=[sent2features(s) for s in train_sents]
y_train=[sent2labels(s) for s in train_sents]
x_test=[sent2features(s) for s in test_sents]
y_test=[sent2labels(s) for s in test_sents]

In [74]:
%%time
crf=sklearn_crfsuite.CRF(
    algorithm='lbfgs',
#     c1=0.20,
#     c2=0.14,
    c1=1,
    c2=2,
    max_iterations=100,
    all_possible_transitions=True
                        )
crf.fit(x_train,y_train)

Wall time: 5.17 s


In [75]:
label_list=list(crf.classes_)
y_pred=crf.predict(x_test)
print(metrics.flat_classification_report(
    y_test, y_pred, label_list, digits=3))

             precision    recall  f1-score   support

          N      0.899     0.963     0.930      3688
          Y      0.745     0.500     0.598       794

avg / total      0.872     0.881     0.871      4482



In [76]:
import joblib
with open('model/CRF_745.mdl','wb') as f:
    joblib.dump(crf,f)

In [73]:
# with open('model/CRF.mdl','rb') as f:
#     crf=joblib.load(f)
print(labels[0])
crf.predict_single(x_train[0])

[('按键', 'n', 3, 'ATT', 'N'), ('反应', 'v', 3, 'ATT', 'N'), ('速度', 'n', 4, 'SBV', 'N'), ('有', 'v', 0, 'HED', 'N'), ('一点', 'm', 6, 'ADV', 'N'), ('慢', 'a', 4, 'VOB', 'N')]


AttributeError: 'NoneType' object has no attribute 'tag'

In [77]:
# # fine tuning param
# # %%time
# crf=sklearn_crfsuite.CRF(algorithm='lbfgs',
#                         max_iterations=100,
#                         all_possible_transitions=True)
# params_space={
#     'c1':np.linspace(0,10,10),
#     'c2':np.linspace(0,10,10),
# }
# f1_score=make_scorer(metrics.flat_f1_score,average='weighted',labels=label_list)
# rs=GridSearchCV(crf,
#                      params_space,
#                      cv=5,
#                      verbose=1,
#                      n_jobs=-1,
# #                      n_iter=100,
# #                      scoring=f1_score
#                )
# rs.fit(x_train,y_train)
# print('best params:', rs.best_params_)
# print('best CV score:', rs.best_score_)
# print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))
# crf = rs.best_estimator_
# y_pred = crf.predict(x_test)
# print(metrics.flat_classification_report(
#     y_test, y_pred, label_list, digits=3
# ))

In [78]:
taiyi_train=pd.read_excel('../data/taiyi/train.xlsx')

In [106]:
with codecs.open('../data/taiyi/valid.txt',encoding='utf-8') as f:
    taiyi_test=[]
    for l in f:
        items=l.strip().split(',')
        taiyi_test.append([items[0],'，'.join(items[1:])])
    

In [107]:
for l in taiyi_test:
    if len(l)>2:
        print(l)

In [110]:
taiyi_test=pd.DataFrame(taiyi_test,columns=['row_id','content'])

In [111]:
taiyi_test[:10]

,row_id,content
0,1,比黑人碳头牙刷好用?
1,2,比之前坏的要好些，亮一些。
2,3,质量很好 用着很舒服 感觉有点小贵啊
3,4,发货速度很快，东西收到后基本上和描述的没什么差别，质量还行，比较实用，就是感觉有点渣脚，上面...
4,5,1、每星期至少两次掉线，一天重启几次也正常；2、5G已坏。
5,6,从中学一直在用的，质量好
6,7,此产品太差劲了，没有任何内包装，直接放在盒子里就过来了。铅笔也是打开过多次，已经很久了的样子...
7,8,色粉笔也太贵了吧，而且包装也有些旧了，后悔买啊，早知到上实品店买
8,9,买了跟老公一人一个，用下来还不错，比较静音，充满电能用半个月。
9,10,亚马逊自营的东西是正品，一贯都不错，东西好坏是产品本身的事情。


In [80]:
taiyi_train[:10]

,row_id,content-评论内容,theme-主题,sentiment_word-情感关键词,sentiment_anls-情感正负面
0,1,收到了，太实惠了，买了一大箱，以后继续购买，送货速度快服务也好,NULL;送货速度;服务;,实惠;快;也好;,1;1;1;
1,2,热水器加热时间太长，安装费太贵，预留太阳能口摆设，根本用不到，没有水位指示器，加满热水的指示...,加热时间;安装费;用户;,太长;太贵;不方便;,-1;-1;-1;
2,3,真**差，我都无语了。眼瞎才买了,NULL;NULL;,差;无语;,-1;-1;
3,4,很满意，因为出差，还没有用，东西装上我看了，非常好,NULL;NULL;,满意;好;,1;1;
4,5,给奶奶买的，效果不错，厂家服务也好，给好评[追评],效果;服务;NULL;,不错;也好;好评;,1;1;1;
5,6,还没用，等用了再来追评吧,NaN,NaN,NaN
6,7,这个中奖有用吗？且不说中奖，这个是我买了这么多一加3t钢化膜中最高清最好看的，没有之一。,钢化膜;NULL;,高清;最好;,1;1;
7,8,不错，家用刚刚好，制冷也不错,NULL;制冷;NULL;,不错;不错;好;,1;1;1;
8,9,电流声真心受不了！而且电脑开机的时候会有爆破声。。。,电流;,受不了;,-1;
9,10,电视挺好，都挺满意,电视;NULL;,挺好;满意;,1;1;


In [81]:
# sent_list=re.split('，|。|？|！',sent)
def sent_split(sent):
    sent_list=re.split('，|。|？|！',sent)
    sent_list=[it  for it in sent_list if len(it)>1]
    return sent_list

In [82]:
sent_list=sent_split(taiyi_train.loc[24][1])
sent_list

['清洁功能挺好', '以前就算刚洗完头发不久就有粉粉的头屑', '用这个洗就一点都没有了']

In [83]:
def sent_compress(sentence):
    sent_list=sent_split(sentence)
    sent_compress_list=[]
    sent_no_compress=[]
    sent_label=[]
    for sent in sent_list:
#         print(sent_compress(sent))
        sent_comp=[]
        sent_pos=segment_pos(sent)
        sent_no_compress.append([it[0]for it in sent_pos])
        sent_feature=sent2features(sent_pos)
        label=crf.predict_single(sent_feature)
        sent_label.append(label)
        for ind,lb in enumerate(label):
            if lb == 'N':
                sent_comp.append(sent_pos[ind][0])
        sent_compress_list.append(sent_comp)
    return (sent_compress_list,sent_no_compress,sent_label)

In [56]:
# sent_compress('第一次用华为手机，特意用了一段时间来评价，包装很严谨，手机速度不错，外观很薄，屏幕大小刚好，黑色很大气，真是高大上，很多功能都不错。下次还会来光顾哦。谢谢客服暮思的耐心讲解')

In [84]:
sent_press=[]
sent_no_press=[]
sent_label=[]
for ind,row in enumerate(taiyi_train[taiyi_train.columns[1]]):
    try:
        result=sent_compress(row)
        sent_press.append(result[0])
        sent_no_press.append(result[1])
        sent_label.append(result[2])
    except:
        sent_press.append([])
        sent_no_press.append([])
        sent_label.append([])
taiyi_train['sent_compress']=sent_press
taiyi_train['sent_no_press']=sent_no_press
taiyi_train['sent_label']=sent_label
# taiyi_train.drop('row_id',axis=1,inplace=True)
taiyi_train.to_csv('taiyi_train_compress.csv',index=None,encoding='utf-8')

In [112]:
sent_press=[]
sent_no_press=[]
sent_label=[]
for ind,row in enumerate(taiyi_test[taiyi_test.columns[1]]):
    try:
        result=sent_compress(row)
        sent_press.append(result[0])
        sent_no_press.append(result[1])
        sent_label.append(result[2])
    except:
        sent_press.append([])
        sent_no_press.append([])
        sent_label.append([])
taiyi_test['sent_compress']=sent_press
taiyi_test['sent_no_press']=sent_no_press
taiyi_test['sent_label']=sent_label
# taiyi_test.drop('row_id',axis=1,inplace=True)
taiyi_test.to_csv('taiyi_test_compress.csv',index=None,encoding='utf-8')